In [ ]:
def runSLADSSimulationOnce(Mask,codePath,ImageSet,SizeImage,StopCondParamsObject,Theta,TrainingInfoObject,TestingInfoObject,Resolution,ImageType,UpdateERDParamsObject,BatchSamplingParamsObject,SavePath,ImNum,ImageExtension,PlotResult,Classify,errorPlot,isRunningParallel,lineScanning):
    sys.path.append('code')
    MeasuredIdxs = np.transpose(np.where(Mask==1))
    UnMeasuredIdxs = np.transpose(np.where(Mask==0))

    ContinuousMeasuredValues = performInitialMeasurements(Mask,simulation,img)
    if (len(ContinuousMeasuredValues) == 0):
        sys.exit('Error!!! No values were returned from initial measurements.')
    if Classify=='2C':
        Threshold = filters.threshold_otsu(ContinuousMeasuredValues)
        print('Threhold found using the Otsu method for 2 Class classification = ' + str(Threshold))
        MeasuredValues = ContinuousMeasuredValues < Threshold
        MeasuredValues = MeasuredValues+0
#    elif Classify=='MC':
        #### Classification function to output NewValues ##################
        # NewValues is the vector of measured values post classification
    elif Classify=='N':
        MeasuredValues=ContinuousMeasuredValues

    # Perform SLADS
    IterNum=0
    NumSamples = np.shape(MeasuredValues)[0]
    StopCondFuncVal=np.zeros(( int((SizeImage[0]*SizeImage[1])*(StopCondParamsObject.MaxPercentage)/100)+10,2 ))
    pixelCount = SizeImage[0]*SizeImage[1]

    with tqdm(total = 100, desc = '% Sampled', leave = True, disable = isRunningParallel) as pbar:
        while (checkStopCondFuncThreshold(StopCondParamsObject,StopCondFuncVal,NumSamples,IterNum,SizeImage) != 1):
            if IterNum==0:
                Mask,MeasuredValues,ERDValues,ReconValues,ReconImage,NewIdxs,MaxIdxsVect=updateERDandFindNewLocationFirst(Mask,MeasuredValues,MeasuredIdxs,UnMeasuredIdxs,Theta,SizeImage,TrainingInfoObject,Resolution,ImageType,NumSamples,UpdateERDParamsObject,BatchSamplingParamsObject,lineScanning,InitialMaskObject) 
            else:
                Mask,MeasuredValues,ERDValues,ReconValues,ReconImage,NewIdxs,MaxIdxsVect=updateERDandFindNewLocationAfter(Mask,MeasuredValues,MeasuredIdxs,UnMeasuredIdxs,Theta,SizeImage,TrainingInfoObject,Resolution,ImageType,UpdateERDParamsObject,BatchSamplingParamsObject,StopCondFuncVal,IterNum,NumSamples,NewIdxs,ReconValues,ReconImage,ERDValues,MaxIdxsVect,lineScanning,InitialMaskObject)
            
            if len(NewIdxs) <= 0: #If no new points are to be scanned, then trigger stop condition
                break
            NewContinuousValues = performMeasurements(NewIdxs,BatchSamplingParamsObject,simulation,img)
            ContinuousMeasuredValues = np.hstack((ContinuousMeasuredValues,NewContinuousValues))
            if Classify=='2C':           
                NewValues = NewContinuousValues > Threshold
                NewValues = NewValues+0
    #        elif Classify=='MC':
                #### Classification function to output NewValues ##################
                # NewValues is the vector of measured values post classification            
            elif Classify=='N':
                NewValues=NewContinuousValues    

            Mask,MeasuredValues,MeasuredIdxs,UnMeasuredIdxs = updateMeasurementArrays(NewIdxs,MaxIdxsVect,Mask,MeasuredValues,MeasuredIdxs,UnMeasuredIdxs,NewValues,BatchSamplingParamsObject,SizeImage,lineScanning)
            NumSamples = np.shape(MeasuredValues)[0]

            StopCondFuncVal=computeStopCondFuncVal(ReconValues,MeasuredValues,StopCondParamsObject,ImageType,StopCondFuncVal,MaxIdxsVect,NumSamples,IterNum,BatchSamplingParamsObject,lineScanning,SizeImage)

            #if PlotResult=='Y' and np.remainder(NumSamples,round(0.01*SizeImage[0]*SizeImage[1])) ==0:
                #print(str(np.round(NumSamples*100/(SizeImage[0]*SizeImage[1]))) + ' Percent Sampled')
            IterNum += 1
            pbar.n = round((NumSamples/pixelCount)*100)
            pbar.refresh()
        np.save(SavePath + 'MeasuredValues', MeasuredValues)
        np.save(SavePath + 'MeasuredIdxs', MeasuredIdxs)
        np.save(SavePath + 'UnMeasuredIdxs', UnMeasuredIdxs)
        np.save(SavePath + 'StopCondFuncVal',StopCondFuncVal)
        np.save(SavePath + 'ContinuousMeasuredValues',ContinuousMeasuredValues)
        savemat(SavePath + 'MeasuredIdxs.mat',dict(MeasuredIdxs=MeasuredIdxs))
        savemat(SavePath + 'MeasuredValues.mat',dict(MeasuredValues=MeasuredValues))
        savemat(SavePath + 'UnMeasuredIdxs.mat',dict(UnMeasuredIdxs=UnMeasuredIdxs))
        savemat(SavePath + 'StopCondFuncVal.mat',dict(StopCondFuncVal=StopCondFuncVal))
        savemat(SavePath + 'ContinuousMeasuredValues.mat',dict(ContinuousMeasuredValues=ContinuousMeasuredValues))

        if errorPlot or (PlotResult=='Y'):
            percentSampled = NumSamples*100/(SizeImage[0]*SizeImage[1])
            Difference,ReconImage = performReconOnce(SavePath,TrainingInfoObject,Resolution,SizeImage,ImageType,MeasuredIdxs,UnMeasuredIdxs,MeasuredValues,img)
            TD = Difference/(SizeImage[0]*SizeImage[1])

            #Moved out plotting code into only function it is used
            MSE = (np.sum((ReconImage.astype("float") - img.astype("float")) ** 2))/(float(ReconImage.shape[0] * ReconImage.shape[1]))
            SSIM = compare_ssim(ReconImage.astype("float"), img.astype("float"))

            if errorPlot:
                resultObject = simulationResults()
                resultObject.initialize()
                resultObject.saveErrorData(MSE, SSIM, TD) #Store resulting error information
                resultObject.saveMask(Mask) #Store the sampled mask
                 
            if PlotResult=='Y': 
                #Set plot formatting
                font = {'size' : 18}
                plt.rc('font', **font)
                f = plt.figure(figsize=(20,8))
                plt.suptitle("MSE: %.2f, SSIM: %.2f, TD: %.2f, Percent Sampled: %.2f" % (MSE, SSIM, TD, percentSampled), fontsize=20, fontweight='bold', y = 0.9)
                ax1 = f.add_subplot(131)
                ax1.imshow(Mask, cmap='gist_heat')
                ax1.set_title('Sampled Mask')
                ax2 = f.add_subplot(132)       
                ax2.imshow(ReconImage, cmap='gist_heat')
                ax2.set_title('Reconstructed Image')
                ax3 = f.add_subplot(133)
                ax3.imshow(img, cmap='gist_heat')
                ax3.set_title('Ground-truth Image')
                plt.savefig(SavePath + '.png')

            #pylab.show()
        sys.path.pop() #Hopefully remove the 'sys.path.append('code') flag at the top of this definition
        if errorPlot:
            return resultObject
